In [1]:
# ============================================================
# Итоговый код: проверка модели на примерах из synth_v21
# 1) Берём случайный пример (gold или wrong) из parquet
# 2) Проверяем модель с эталоном
# 3) Дополнительно: прогоняем на кастомном "похожем неверном" решении для задачи из датасета
# ============================================================

import os
import re
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ.setdefault("HF_HUB_DISABLE_XET", "1")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ---------------------------
# CONFIG
# ---------------------------
MERGED_DIR = r".\outputs2\ege_checker_qwen2p5_0p5b_lora_synth_v21\merged_model"
DATA_PARQUET = r".\synth_v21\all_problems_balanced_synth_v21.parquet"

MAX_NEW_TOKENS = 220
SEED = 42


# ---------------------------
# Load tokenizer/model
# ---------------------------
def load_tokenizer(path: str):
    try:
        tok = AutoTokenizer.from_pretrained(path, use_fast=True, fix_mistral_regex=True)
    except TypeError:
        tok = AutoTokenizer.from_pretrained(path, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
    return tok

def load_model_and_tokenizer(merged_dir: str):
    dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else torch.float16
    tokenizer = load_tokenizer(merged_dir)
    try:
        model = AutoModelForCausalLM.from_pretrained(merged_dir, dtype=dtype, device_map="auto")
    except TypeError:
        model = AutoModelForCausalLM.from_pretrained(merged_dir, torch_dtype=dtype, device_map="auto")
    model.eval()
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer(MERGED_DIR)
print("Loaded OK. dtype:", next(model.parameters()).dtype, "device:", model.device)
print("Tokenizer:", tokenizer.__class__.__name__, "| pad_token_id:", tokenizer.pad_token_id)


# ---------------------------
# Prompt + parsing + generate
# ---------------------------
SYSTEM_PROMPT = (
    "Ты — эксперт ЕГЭ по математике и проверяешь решения.\n"
    "Тебе дано условие задачи, эталонное решение и решение ученика.\n"
    "Сравни решение ученика с эталоном и оцени корректность логики и ответа.\n\n"
    "Формат ответа строго:\n"
    "Вердикт: верно|неверно\n"
    "Пояснение: 2–6 предложений. Если неверно — укажи конкретный шаг/место и почему это ошибка."
)

VERDICT_RE = re.compile(r"^вердикт\s*:\s*(верно|неверно)\b", re.IGNORECASE | re.MULTILINE)

def extract_verdict(text: str):
    m = VERDICT_RE.search(text or "")
    if not m:
        return None
    return 1 if m.group(1).lower() == "верно" else 0

def build_user_prompt(condition: str, student_solution: str, reference_solution: str, answer_hint: str | None):
    parts = [
        "Условие задачи:\n" + (condition or "").strip(),
        "Эталонное решение (для проверки):\n" + (reference_solution or "").strip(),
        "Решение ученика:\n" + (student_solution or "").strip(),
    ]
    if answer_hint is not None and str(answer_hint).strip():
        parts.append("Ожидаемый ответ (справочно): " + str(answer_hint).strip())

    parts.append(
        "Задание: сравни решение ученика с эталоном. "
        "Если есть ошибка — укажи конкретное место/шаг и почему это ошибка. "
        "Ответ дай строго в формате:\n"
        "Вердикт: верно|неверно\n"
        "Пояснение: ..."
    )
    return "\n\n".join(parts).strip()

@torch.no_grad()
def generate_assistant_only(messages, max_new_tokens=MAX_NEW_TOKENS):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    gen_ids = out[0, inputs["input_ids"].shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

def check_solution(condition: str, student_solution: str, reference_solution: str, answer_hint: str | None = None):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(condition, student_solution, reference_solution, answer_hint)},
    ]
    assistant_text = generate_assistant_only(messages)
    return {
        "assistant_text": assistant_text,
        "verdict": extract_verdict(assistant_text),
        "messages": messages,
    }


# ---------------------------
# Dataset helpers
# ---------------------------
df = pd.read_parquet(DATA_PARQUET)
print("df loaded:", df.shape)
print("splits:", df["split"].value_counts().to_dict())
print("labels:", df["label"].value_counts().to_dict())

rng = np.random.default_rng(SEED)

def sample_from_df(df: pd.DataFrame, split="val", label=0):
    sub = df[(df["split"] == split) & (df["label"] == label)].copy()
    if sub.empty:
        raise ValueError("Empty subset: check split/label filters")
    idx = int(rng.integers(0, len(sub)))
    return sub.iloc[idx]

def print_example_row(row: pd.Series, max_chars=3500):
    uid = row["row_uid"]
    tag = row["error_tag"]
    label = int(row["label"])
    ans = row.get("answer_norm", None)
    def clip(s): 
        s = str(s or "").strip()
        return s if len(s) <= max_chars else (s[:max_chars] + "\n...[TRUNCATED]...")
    print("\n================ EXAMPLE ================")
    print("uid:", uid, "| split:", row["split"], "| label:", label, "| tag:", tag, "| answer_norm:", ans)
    print("----------------------------------------")
    print("CONDITION:\n", clip(row["condition_for_train"]))
    print("----------------------------------------")
    print("STUDENT:\n", clip(row["student_solution"]))
    print("----------------------------------------")
    print("REF:\n", clip(row["solution_ref"]))
    print("========================================\n")


# ============================================================
# TEST 1: случайный WRONG из val (label=0)
# ============================================================
row_wrong = sample_from_df(df, split="val", label=0)
print_example_row(row_wrong)

res_wrong = check_solution(
    condition=str(row_wrong["condition_for_train"]),
    student_solution=str(row_wrong["student_solution"]),
    reference_solution=str(row_wrong["solution_ref"]),
    answer_hint=str(row_wrong.get("answer_norm", "")),
)
print("MODEL VERDICT:", res_wrong["verdict"])
print(res_wrong["assistant_text"])


# ============================================================
# TEST 2: случайный GOLD из val (label=1)
# ============================================================
row_gold = sample_from_df(df, split="val", label=1)
print_example_row(row_gold)

res_gold = check_solution(
    condition=str(row_gold["condition_for_train"]),
    student_solution=str(row_gold["student_solution"]),
    reference_solution=str(row_gold["solution_ref"]),
    answer_hint=str(row_gold.get("answer_norm", "")),
)
print("MODEL VERDICT:", res_gold["verdict"])
print(res_gold["assistant_text"])


# ============================================================
# TEST 3: кастомный неверный пример
# ============================================================
condition_text = """
Хорда AB делит окружность на две части, градусные величины которых относятся как 5 : 7.
Под каким углом видна эта хорда из точки C, принадлежащей меньшей дуге окружности?
Ответ дайте в градусах.
""".strip()

reference_solution_text = """
Решение. Из точки C хорда АВ видна под углом АCВ. Пусть большая часть окружности равна 7x, тогда меньшая равна 5x.

7x + 5x = 360°  ⇒  12x = 360°  ⇒  x = 30°.

Значит, меньшая дуга окружности равна 150°, а большая — 210°.
Вписанный угол равен половине дуги, на которую он опирается, значит угол ACB (точка C на меньшей дуге) опирается на большую дугу 210°,
поэтому ∠ACB = 210° / 2 = 105°.

Ответ: 105.
""".strip()

student_solution_text_wrong = """
Решение. Из точки C хорда AB видна под углом ACB. Пусть большая часть окружности равна 7x, тогда меньшая равна 5x.

7x + 5x = 360°  ⇒  12x = 360°  ⇒  x = 30°.

Значит, меньшая дуга окружности равна 150°, а большая — 210°.
Вписанный угол равен половине дуги, на которую он опирается. Так как точка C лежит на меньшей дуге, то угол ACB опирается на меньшую дугу 150°,
следовательно, ∠ACB = 150° / 2 = 75°.

Ответ: 75.
""".strip()

print("\n================ CUSTOM WRONG (CHORD) ================")
res_custom = check_solution(
    condition=condition_text,
    student_solution=student_solution_text_wrong,
    reference_solution=reference_solution_text,
    answer_hint="105",
)
print("MODEL VERDICT:", res_custom["verdict"])
print(res_custom["assistant_text"])


Loaded OK. dtype: torch.bfloat16 device: cuda:0
Tokenizer: Qwen2TokenizerFast | pad_token_id: 151643


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


df loaded: (20356, 10)
splits: {'train': 20036, 'val': 320}
labels: {1: 10178, 0: 10178}

================ EXAMPLE ================
uid: a5d8305366f40121 | split: val | label: 0 | tag: wrong_final_answer_replace | answer_norm: 1,96
----------------------------------------
CONDITION:
 На рисунке изображён график функции f левая круглая скобка x правая круглая скобка =k корень из: начало аргумента: x конец аргумента . Найдите значение x, при котором f левая круглая скобка x правая круглая скобка =3,5.
----------------------------------------
STUDENT:
 Решение. По графику, f(4) = 5, тогда k умножить на корень из: начало аргумента: 4 конец аргумента =5 равносильно 2k=5 равносильно k=2,5. Таким образом, 

2,5 умножить на корень из: начало аргумента: x конец аргумента =3,5 равносильно корень из: начало аргумента: x конец аргумента =1,4 равносильно x=1,96.

Ответ: 0,96.
----------------------------------------
REF:
 Решение. По графику, f(4) = 5, тогда k умножить на корень из: начало аргумент

C:\Users\narumaru\anaconda3\Lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


MODEL VERDICT: 0
Вердикт: неверно
Пояснение: В вычислениях получается один результат, но в конце указан другой ответ. Итоговый ответ не следует из приведённых преобразований.

================ EXAMPLE ================
uid: 5057e7eda18d9d46 | split: val | label: 1 | tag: gold | answer_norm: 2
----------------------------------------
CONDITION:
 Найдите площадь четырехугольника, изображенного на клетчатой бумаге с размером клетки 1 см \times 1 см (см. рис.). Ответ дайте в квадратных сантиметрах.
----------------------------------------
STUDENT:
 Решение. Площадь четырёхугольника равна разности площади большого квадрата и двух прямоугольных треугольников, гипотенузы которых являются сторонами исходного четырехугольника. Поэтому 

S=2 умножить на 2 минус \\frac{1,}{2 конец дроби умножить на 1 умножить на 2 минус \\frac{1,}{2 конец дроби умножить на 1 умножить на 2=2 см в квадрате .}}

Ответ: 2.
----------------------------------------
REF:
 Решение. Площадь четырёхугольника равна разности 